# 데이터 결합 및 부분 선택

### 주요 내용

1. 데이터 결합
2. index, columns을 활용한 부분 선택 
3. 조건을 활용한 관측치 선택

<br>

### 목표 
1. 복수의 데이터를 적절한 방법으로 결합할 수 있다.
2. 변수 이름 등을 활용하여 부분 데이터를 선택한다.
3. 주제에 맞게 조건을 활용하여 부분 관측치를 선택한다. 


<br>
<hr>
<br>

## 1. DataFrame 형식의 활용

pandas는 데이터를 저장하는 형식 **DataFrame**을 중심으로 구성되어 있음  
pandas의 다양한 함수를 활용해서 데이터를 불러오거나 저장할 수 있고, 분석 과정에서 필요한 전처리나 집계 작업도 가능


In [2]:
# 라이브러리 불러오기
import pandas as pd

<br>

DataFrame에서 각각의 열, 변수가 하나의 Series로 저장되어 있음  
Series의 메서드와 DataFrame의 메서드 구분 필요  

만약 직접 DataFrame을 만들어야 할 때는 *DataFrame( )* 과 딕셔너리를 활용

In [1]:
# 딕셔너리를 활용한 DataFrame 생성
df_own = pd.DataFrame(
    
    {'FIRST' : ['A', 'B', 'C', 'D'],
     'SECOND': [7,6,5,8], 
     'THIRD' : pd.date_range('2022-12-05', periods=4, freq='W-SUN')}

) # freq='W-MON' : 매주 월요일
df_own

NameError: name 'pd' is not defined

<br>
<hr>
<br>

## 2. 데이터 결합

### 2.1. concat( )을 활용한 동일 구조 데이터 행 결합

구조는 똑같고 기간이나 상품만 다른 여러 데이터가 있으면 pandas의 *concat()* 으로 결합해서 활용  
함수 안에서 `axis=0`옵션을 활용해서 행 결합(아래로 이어 붙이기)을 할 수 있고, `axis=1`로 열 결합도 가능  
`axis=0`이 기본값며 생략 가능

In [4]:
# 행 결합
    ## 출처 : 국토교통부 실거래가(http://rtdown.molit.go.kr/)
df_apt1 = pd.read_csv('data/아파트(매매)__실거래가_20210902153616.csv', skiprows=15, encoding='CP949')
df_apt1

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일
0,서울특별시 강남구 개포동,1282,1282,0,개포래미안포레스트,59.9200,202108,21,"199,500",6,2020,개포로 264,NaN
1,서울특별시 강남구 개포동,185,185,0,개포주공 7단지,83.7000,202108,20,"280,000",2,1983,개포로 516,NaN
2,서울특별시 강남구 개포동,138,138,0,디에이치아너힐즈,59.8732,202108,17,"233,000",4,2019,삼성로 11,NaN
3,서울특별시 강남구 개포동,1280,1280,0,래미안블레스티지,59.9670,202108,14,"227,000",10,2019,선릉로 8,NaN
4,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,49.8600,202108,1,"169,000",10,1992,개포로109길 9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,서울특별시 강남구 청담동,14,14,0,"청담2차 이-편한세상(201동,202동,203동)",84.9700,202108,14,"200,000",7,2006,도산대로70길 25,NaN
232,서울특별시 강남구 청담동,15-21,15,21,청담2차 이-편한세상(205동),119.5500,202108,9,"215,000",3,2006,도산대로78길 51,NaN
233,서울특별시 강남구 청담동,34-13,34,13,청담스위트,13.4250,202108,17,"22,000",4,2015,학동로73길 13,NaN
234,서울특별시 강남구 청담동,134-38,134,38,청담자이,89.3560,202108,4,"400,000",16,2011,영동대로138길 12,NaN


In [5]:
df_apt2 = pd.read_csv('data/아파트(매매)__실거래가_20210902153636.csv', skiprows=15, encoding='CP949')
df_apt2

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일
0,서울특별시 서초구 내곡동,BL-1,1,0,서초더샵포레,114.7100,202108,14,"185,000",8,2014,헌릉로8길 58,NaN
1,서울특별시 서초구 반포동,18-1,18,1,래미안퍼스티지,169.3100,202108,7,"527,000",26,2009,반포대로 275,NaN
2,서울특별시 서초구 반포동,757,757,0,반포 주공1단지,106.2500,202108,9,"455,000",2,1973,신반포로 9,NaN
3,서울특별시 서초구 반포동,1341,1341,0,반포래미안아이파크,99.9200,202108,5,"350,000",3,2018,서초중앙로 220,NaN
4,서울특별시 서초구 반포동,1342,1342,0,반포써밋,84.9716,202108,7,"300,000",20,2018,고무래로 89,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,서울특별시 서초구 잠원동,160,160,0,신반포자이,59.8600,202108,5,"255,000",6,2018,잠원로 60,NaN
70,서울특별시 서초구 잠원동,63-2,63,2,신반포청구,84.8600,202108,6,"239,000",8,1998,신반포로33길 66,NaN
71,서울특별시 서초구 잠원동,159,159,0,아크로리버뷰신반포,78.5000,202108,4,"290,000",3,2018,잠원로 117,NaN
72,서울특별시 서초구 잠원동,159,159,0,아크로리버뷰신반포,78.5000,202108,4,"290,000",3,2018,잠원로 117,20210826.0


In [8]:
df_apt3 = pd.read_csv('data/아파트(매매)__실거래가_20210902153655.csv', skiprows=15, encoding='CP949')
df_apt3

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일1
0,서울특별시 송파구 가락동,Jan-95,95,1,가락금호아파트,59.91,202108,8,"123,000",1,1997,송파대로32길 15,NaN
1,서울특별시 송파구 가락동,06월 21일,21,6,가락쌍용(2차),59.88,202108,2,"121,000",2,1999,송이로15길 31,NaN
2,서울특별시 송파구 가락동,06월 21일,21,6,가락쌍용(2차),84.42,202108,12,"135,000",1,1999,송이로15길 31,NaN
3,서울특별시 송파구 가락동,70-19,70,19,대림,84.11,202108,9,"140,000",2,1988,송이로 88,NaN
4,서울특별시 송파구 가락동,70-19,70,19,대림,120.96,202108,11,"170,000",5,1988,송이로 88,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,서울특별시 송파구 풍납동,220-2,220,2,신동아파밀리에,59.36,202108,1,"85,900",8,1993,풍성로6길 15,NaN
72,서울특별시 송파구 풍납동,510,510,0,신성노바빌아파트,59.78,202108,16,"98,500",15,2000,한가람로 468,NaN
73,서울특별시 송파구 풍납동,401-1,401,1,쌍용,84.85,202108,11,"143,000",15,1994,올림픽로47길 12,NaN
74,서울특별시 송파구 풍납동,508,508,0,한강극동,84.76,202108,25,"111,000",11,1995,토성로 38-6,NaN


In [10]:
df_apt = pd.concat([df_apt1, df_apt2, df_apt3], join="inner")
df_apt

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1282,1282,0,개포래미안포레스트,59.9200,202108,21,"199,500",6,2020,개포로 264
1,서울특별시 강남구 개포동,185,185,0,개포주공 7단지,83.7000,202108,20,"280,000",2,1983,개포로 516
2,서울특별시 강남구 개포동,138,138,0,디에이치아너힐즈,59.8732,202108,17,"233,000",4,2019,삼성로 11
3,서울특별시 강남구 개포동,1280,1280,0,래미안블레스티지,59.9670,202108,14,"227,000",10,2019,선릉로 8
4,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,49.8600,202108,1,"169,000",10,1992,개포로109길 9
...,...,...,...,...,...,...,...,...,...,...,...,...
71,서울특별시 송파구 풍납동,220-2,220,2,신동아파밀리에,59.3600,202108,1,"85,900",8,1993,풍성로6길 15
72,서울특별시 송파구 풍납동,510,510,0,신성노바빌아파트,59.7800,202108,16,"98,500",15,2000,한가람로 468
73,서울특별시 송파구 풍납동,401-1,401,1,쌍용,84.8500,202108,11,"143,000",15,1994,올림픽로47길 12
74,서울특별시 송파구 풍납동,508,508,0,한강극동,84.7600,202108,25,"111,000",11,1995,토성로 38-6


<br>

> **DataFrame**에서 행 번호에 해당하는 **index**는 중요한 역할을 합니다.  
예를 들어 아래처럼 index를 확인할 수 있고, 특정 index를 지정해서 관측치를 선택하는 것도 가능합니다. 

In [11]:
df_apt.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
            ...
            66, 67, 68, 69, 70, 71, 72, 73, 74, 75],
           dtype='int64', length=386)

In [12]:
# index 0 관측치 선택
df_apt.loc[0]

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1282,1282,0,개포래미안포레스트,59.92,202108,21,"199,500",6,2020,개포로 264
0,서울특별시 서초구 내곡동,BL-1,1,0,서초더샵포레,114.71,202108,14,"185,000",8,2014,헌릉로8길 58
0,서울특별시 송파구 가락동,Jan-95,95,1,가락금호아파트,59.91,202108,8,"123,000",1,1997,송파대로32길 15


결합 이전 기존 Index 활용으로 **0** 인덱스 관측치의 중복 발생  
행 결합이나 정렬 이후 인덱스를 재지정하거나 초기화 필요 

In [13]:
# reset_index()을 활용한 index 초기화
    ## drop=True: 기존 인덱스를 변수로 추가할 지 버릴지 선택
df_apt = df_apt.reset_index(drop=True)
df_apt

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명
0,서울특별시 강남구 개포동,1282,1282,0,개포래미안포레스트,59.9200,202108,21,"199,500",6,2020,개포로 264
1,서울특별시 강남구 개포동,185,185,0,개포주공 7단지,83.7000,202108,20,"280,000",2,1983,개포로 516
2,서울특별시 강남구 개포동,138,138,0,디에이치아너힐즈,59.8732,202108,17,"233,000",4,2019,삼성로 11
3,서울특별시 강남구 개포동,1280,1280,0,래미안블레스티지,59.9670,202108,14,"227,000",10,2019,선릉로 8
4,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,49.8600,202108,1,"169,000",10,1992,개포로109길 9
...,...,...,...,...,...,...,...,...,...,...,...,...
381,서울특별시 송파구 풍납동,220-2,220,2,신동아파밀리에,59.3600,202108,1,"85,900",8,1993,풍성로6길 15
382,서울특별시 송파구 풍납동,510,510,0,신성노바빌아파트,59.7800,202108,16,"98,500",15,2000,한가람로 468
383,서울특별시 송파구 풍납동,401-1,401,1,쌍용,84.8500,202108,11,"143,000",15,1994,올림픽로47길 12
384,서울특별시 송파구 풍납동,508,508,0,한강극동,84.7600,202108,25,"111,000",11,1995,토성로 38-6


In [19]:
# index 0 관측치 재선택
df_apt.loc[0]

시군구         서울특별시 강남구 개포동
번지                   1282
본번                   1282
부번                      0
단지명             개포래미안포레스트
전용면적(㎡)             59.92
계약년월               202108
계약일                    21
거래금액(만원)          199,500
층                       6
건축년도                 2020
도로명               개포로 264
Name: 0, dtype: object

#### [실습]  데이터 결합 및 인덱스 초기화

출처 : [서울시 지하철 호선별 역별 승하차 인원수](http://data.seoul.go.kr/dataList/OA-12914/S/1/datasetView.do)

1. `data`폴더의 `CARD_SUBWAY_MONTH_`로 시작하는 3개 데이터 확인하기  
    


2. 1.의 데이터를 각각 불러와서 저장하고, pd.concat()으로 행 결합하기(encoding='CP949' 활용)


3. index 초기화 하기



In [26]:
df_card1 = pd.read_csv("./data/CARD_SUBWAY_MONTH_201907.csv", encoding='cp949')
df_card2 = pd.read_csv("./data/CARD_SUBWAY_MONTH_202007.csv", encoding='cp949')
df_card3 = pd.read_csv("./data/CARD_SUBWAY_MONTH_202107.csv", encoding='cp949')

In [29]:
df_card = pd.concat([df_card1,df_card2,df_card3]).reset_index(drop=True)

In [3]:
pd.read_csv("../../CARD_SUBWAY_MONTH_202305.csv")

,사용일자,노선명,역명,승차총승객수,하차총승객수,등록일자
20230501,8호선,산성,4066,4316,20230504,NaN
20230501,7호선,신대방삼거리,11093,10879,20230504,NaN
20230501,6호선,태릉입구,4641,5122,20230504,NaN
20230501,6호선,석계,9777,10288,20230504,NaN
20230501,6호선,돌곶이,7213,6895,20230504,NaN
...,...,...,...,...,...,...
20230531,2호선,봉천,26698,25052,20230603,NaN
20230531,2호선,서울대입구(관악구청),51435,50092,20230603,NaN
20230531,2호선,낙성대(강감찬),30181,29309,20230603,NaN
20230531,2호선,사당,41501,45490,20230603,NaN


#### [참고] glob과 for 반복문을 활용한 복수 데이터 처리

**glob** 라이브러리의 *glob()* 을 활용하면 복수의 데이터 경로를 손쉽게 처리 가능

In [30]:
# 대상 파일 목록 생성
from glob import glob

file_list  = glob('data/apt/*.csv')
file_list

['data/apt\\아파트(매매)__실거래가_20220930082940.csv',
 'data/apt\\아파트(매매)__실거래가_20220930083006.csv',
 'data/apt\\아파트(매매)__실거래가_20220930083033.csv',
 'data/apt\\아파트(매매)__실거래가_20220930083102.csv',
 'data/apt\\아파트(매매)__실거래가_20220930083124.csv',
 'data/apt\\아파트(매매)__실거래가_20220930083151.csv',
 'data/apt\\아파트(매매)__실거래가_20220930083220.csv',
 'data/apt\\아파트(매매)__실거래가_20220930083248.csv']

In [31]:
# for를 활용한 반복
a = list()
for path_ in file_list:
    a.append(pd.read_csv(path_, skiprows=15, encoding='CP949'))
a

[               시군구      번지   본번  부번               단지명   전용면적(㎡)    계약년월  계약일  \
 0    서울특별시 강남구 개포동      12   12   0           삼익대청아파트   39.5300  202208    8   
 1    서울특별시 강남구 개포동      12   12   0        성원대치2단지아파트   49.8600  202208   10   
 2    서울특별시 강남구 개포동      12   12   0        성원대치2단지아파트   49.8600  202208   31   
 3    서울특별시 강남구 논현동    58-2   58   2            마일스디오빌   36.2900  202208    6   
 4    서울특별시 강남구 논현동    58-2   58   2            마일스디오빌   36.2900  202208    8   
 5    서울특별시 강남구 논현동    58-2   58   2            마일스디오빌   36.2900  202208   12   
 6    서울특별시 강남구 논현동    58-2   58   2            마일스디오빌   36.2900  202208   12   
 7    서울특별시 강남구 논현동    58-2   58   2            마일스디오빌   36.2900  202208   27   
 8    서울특별시 강남구 논현동    58-2   58   2            마일스디오빌   36.2900  202208   27   
 9    서울특별시 강남구 논현동   221-7  221   7        한양수자인어반게이트   16.2000  202208    4   
 10   서울특별시 강남구 대치동     503  503   0             개포우성1   84.8100  202208   30   
 11   서울특별시 강남구 대치동     888 

In [32]:
# 최종 작업
df_subway = pd.concat(a).reset_index(drop=True)
df_subway

,시군구,번지,본번,부번,단지명,전용면적(㎡),계약년월,계약일,거래금액(만원),층,건축년도,도로명,해제사유발생일,거래유형,중개사소재지
0,서울특별시 강남구 개포동,12,12,0,삼익대청아파트,39.53,202208,8,"119,000",12,1992,개포로109길 21,NaN,중개거래,서울 강남구
1,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,49.86,202208,10,"133,000",1,1992,개포로109길 9,NaN,중개거래,서울 강남구
2,서울특별시 강남구 개포동,12,12,0,성원대치2단지아파트,49.86,202208,31,"136,000",14,1992,개포로109길 9,NaN,중개거래,"서울 강남구, 서울 서초구"
3,서울특별시 강남구 논현동,58-2,58,2,마일스디오빌,36.29,202208,6,"38,500",11,2004,학동로 165,NaN,중개거래,"서울 강남구, 서울 서초구"
4,서울특별시 강남구 논현동,58-2,58,2,마일스디오빌,36.29,202208,8,"38,000",14,2004,학동로 165,NaN,중개거래,서울 강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,서울특별시 강남구 자곡동,619,619,0,엘에이치강남힐스테이트,51.99,202201,8,"103,000",7,2015,자곡로3길 21,20220502.0,중개거래,서울 강남구
615,서울특별시 강남구 자곡동,619,619,0,엘에이치강남힐스테이트,51.96,202201,17,"118,000",11,2015,자곡로3길 21,NaN,중개거래,서울 강남구
616,서울특별시 강남구 청담동,76-12,76,12,라테라스청담,18.62,202201,17,"34,000",8,2018,학동로81길 9,NaN,중개거래,서울 강남구
617,서울특별시 강남구 청담동,10,10,0,삼환아파트101동,84.91,202201,17,"170,000",2,1999,학동로77길 49,NaN,중개거래,서울 강남구


In [ ]:
a = [pd.read_csv(path_, skiprows=15, encoding='CP949') for path_ in glob('data/apt/*.csv')]
df_subway = pd.concat(a).reset_index(drop=True)

<br>

### 2.2. merge()를 활용한 KEY 변수 기준 결합 

SQL의 JOIN, Excel의 VLOOKUP()과 같이 KEY 변수를 활용한 데이터 결합은 *merge()* 를 활용

In [ ]:
# 예제 데이터 불러오기
df_left  = pd.read_csv('data/data_left.csv')
df_right = pd.read_csv('data/data_right.csv')

In [ ]:
df_left

In [ ]:
df_right

<br>

> key를 활용한 데이터 결합에서는 일치하는 key가 있는, 짝이 있는 관측치만 출력하는 것이 기본값으로 설정되어 있습니다. SQL에서는 이것을 **inner join**이라고 부릅니다.  

*merge()* 에서 `how=` 옵션을 활용해서 다음과 같은 데이터 결합 방법 지정 

+ `inner`: inner join. key 기준 일치하는 관측치만 포함
+ `left`:  left join. inner join의 결과물과 왼쪽 데이터의 짝 없는 관측치 포함
+ `right`: right join. inner join의 결과물과 오른쪽 데이터의 짝 없는 관측치 포함
+ `outer`: full outer join. inner join과 양쪽 데이터의 짝이 없는 모든 관측치 포함

In [ ]:
# merge()를 활용한 결합
pd.merge(df_left, df_right, how='inner', on='category')

In [ ]:
# left join
pd.merge(df_left, df_right, how='left', on='category')

In [ ]:
# right join
pd.merge(df_left, df_right, how='right', on='category')

In [ ]:
# full outer join
pd.merge(df_left, df_right, how='outer', on='category')

<br>
<hr>
<br>


## 3. 데이터 부분 선택

일반적인 비즈니스 데이터 분석에서 주제와 기간, 사이트, 제품, 공정 등 본인의 업무와 관련이 있는 일부 데이터만 선택하고 활용  
SQL을 활용한 데이터 추출 과정과 별개로 Python에서 각 분석 과정에서 맞게 부분 데이터를 다시 선택하고 사용

<br> 

In [ ]:
# 예제 데이터 불러오기
import pandas as pd
df_ins = pd.read_csv('data/insurance.csv')
df_ins.head()

<br>

### 3.1. .을 활용한 변수 선택

DataFrame 뒤에 마침표(.)를 찍고 `Tab` 키를 눌러 DataFrame의 메서드들과 함께 변수이름을 확인 가능  
.은 가장 간단한 변수 선택 방법이며 선택된 변수는 **Series** 형식으로 출력  

In [ ]:
# .을 활용한 하나의 변수 선택
df_ins.age

<br>


### 3.2. 대괄호를 활용한 데이터 부분 선택

DataFrame에 대괄호를 붙이고 슬라이스:로 관측치 번호를 지정하거나 따옴표''로 변수 이름을 넣어 데이터 부분을 선택 가능  
변수 이름을 리스트 형식으로 묶어 넣어 여러개 변수를 한번에 선택 가능

In [ ]:
# 관측치 선택
df_ins[0:3]

In [ ]:
# 한 변수 선택 
df_ins['age']

In [ ]:
# 리스트를 활용한 복수 변수 선택
df_ins[['age','smoker','charges']]

In [ ]:
# 연속된 대괄호 활용가능
df_ins[0:5][['age','smoker','charges']]

<br>

#### [실습]  

1. 아래의 명령어를 실행해서 df_subway 데이터 생성하기 

2. .columns 메서드를 활용해서 변수이름 확인하기

3. 슬라이스를 활용하여 11~15번째 관측치 선택하기

4. '사용일자', '역명', '하차총승객수' 세 변수 선택하기



    

In [ ]:
df_subway = pd.read_csv('./data/CARD_SUBWAY_MONTH_202107.csv', encoding='CP949')
df_subway

<br>

## 3.3. loc과 iloc을 활용한 관측치/변수 선택

loc은 행 이름(index)과 열 이름(column)으로 데이터에서 일부를 선택하고, iloc은 정수(integer) 형식의 행 번호, 열 번호를 활용  
두 방법 모두 리스트[ ]나 슬라이스:를 활용한 방법을 지원



In [ ]:
# 실습을 위해 원본 데이터를 복제(copy)하고 부분선택
df_ins2 = df_ins.copy()[0:10]
df_ins2

In [ ]:
# 실습을 위해 인덱스를 별도로 지정
df_ins2['idx'] = list(range(101, 111))
df_ins2.set_index('idx', inplace=True)
df_ins2

<br> 

### 3.3.1. loc을 활용한 부분 선택

loc은 실제로 눈에 보이는 index와 column을 활용

In [ ]:
df_ins2.loc[101]

In [ ]:
df_ins2.loc[[101, 103]]

In [ ]:
df_ins2.loc[101:103]

In [ ]:
df_ins2.loc[101:103, 'smoker']

In [ ]:
# 변수이름 리스트 활용가능
df_ins2.loc[101:103, ['smoker','region']]

In [ ]:
# 변수이름 슬라이스:를 활용 가능 
df_ins2.loc[101:103, 'smoker':'charges']

In [ ]:
# 모든 관측치 선택할 때는 :
df_ins2.loc[:, 'smoker':'charges']

<br> 

### 3.2.2. iloc을 활용한 부분 선택

iloc은 이름과 상관없이 정수로 표현한 위치, 번호를 활용하며 리스트나 슬라이스 활용 방법은 loc과 동일

In [ ]:
df_ins2.iloc[0:3, [0,3,4]]

#### [실습] 

1. df_pr에서 index 기준 '3'의 'Weight' 확인하기
2. df_pr에서 index 기준 '11~15'의 'Age'부터 'Exercise'까지 선택하기
3. df_pr에서 첫번째 ~ 다섯번째 관측치와 다섯번째 ~ 열번째 변수 선택하기

In [ ]:
df_pr = pd.read_csv('data/PulseRates.csv')
df_pr.head()

### 3.4. 함수를 활용한 여러 변수 선택 



In [ ]:
# filter( ) 메서드에서 변수 이름 패턴을 활용한 선택 
df_ins.filter(regex='^s')
    ## regex :  정규표현식(regular expression)
    ## '^s' : 's'로 시작하는 이름/텍스트
    

In [ ]:
# 변수형식 확인하기
df_ins.dtypes
    ## int/float : 숫자
    ## object : 문자열

In [ ]:
# 수치형 변수만 선택
df_ins.select_dtypes(include='number')

In [ ]:
# 문자열 변수만 선택
df_ins.select_dtypes(include='object')

<br>

#### [실습] Student performance 데이터 활용

1. df_sp에서 수치형 변수만 선택
2. df_sp에서 문자열 변수만 선택
3. df_sp에서 이름에 'score'가 들어간 변수만 선택


In [ ]:
df_sp = pd.read_csv('data/StudentsPerformance.csv')
df_sp.head()

<br>

### 3.5. 조건을 활용한 관측치 선택

SQL에서 WHERE 절이나 Excel의 Filter와 같이 데이터에서 부분을 선택할 때 조건을 활용하는 경우 많음  
[ ]나 .loc[ ] 안에 조건식을 넣어서 조건과 일치하는 관측치만 선택 가능

In [ ]:
# 1 단계 : 조건 설정(결과는 True/False)
    # bool 타입 Series 
df_ins['age'] < 30

In [ ]:
# 2 단계 : []와 조건을 활용한 관측치 선택
df_ins[df_ins['age'] < 30]

In [ ]:
# &와 |를 활용한 조건 결합
df_ins[(df_ins['age'] < 30) & (df_ins['sex'] == 'female')]

In [ ]:
df_ins[(df_ins['age'] < 30) | (df_ins['sex'] == 'female')]

<br> 

> 특히 비즈니스 데이터는 범주화, 그룹화된 변수들이 많고, 수많은 담당자들이 그 중 일부 범주, 그룹, 수준을 나눠서 운영하는 경우가 많습니다.  
*isin()* 을 활용해서 내가 관심있는 범주인지 아닌지 포함여부에 대한 연산이 가능합니다.

In [ ]:
# 변수 region의 수준 목록 확인 및 관심 수준 선택
df_ins['region'].unique()

In [ ]:
# isin()을 활용한 특정 수준 관측치 선택
cond1 = df_ins['region'].isin(['southeast','northwest'])
cond1

In [ ]:
df_ins[cond1]

<br>

#### [실습]

1. df_sp에서 math score가 90 이상인 관측치 선택
2. df_sp에서 race/ethnicity가 'group D', 'group E'인 관측치 선택(isin() 활용)
3. 1.과 2.를 동시에 만족하는 관측치 선택 

In [ ]:
df_sp.head()

#### [참고] Series의 str 메서드 활용
문자열 Series(한 변수)에서 str 함수를 활용하면 특정 단어를 포함하거나 특정 패턴과 일치하는 관측치를 선택 가능

In [ ]:
df_sp['parental level of education'].str.startswith('b')

In [ ]:
df_sp['parental level of education'].str.endswith('college')

In [ ]:
df_sp['parental level of education'].str.contains('degree')

<br>

#### [참고] Series의 between 메서드 활용
수치형 Series(한 변수)에서 *between()* 으로 특정 범위 내 관측치 선택 가능

In [ ]:
df_sp['math score'].between(80, 89.9)

In [ ]:
# 양쪽 끝 경계 포함 여부 지정 가능
    # 'both', 'left', 'right'
df_sp[df_sp['math score'].between(80, 90, inclusive='left')] 

<br>

#### [참고] ~를 활용한 부정(True/False 반전)
bool Series(True/False) 앞에 **~** 를 붙여서 True와 False를 뒤집기 가능

In [ ]:
cond1 = df_sp['math score'].between(80, 90, inclusive='left')
cond1

In [ ]:
~cond1

In [ ]:
df_sp[~cond1]

<br>

### 3.6. 함수를 활용한 부분 관측치 선택


In [ ]:
# head( )와 tail()
df_ins.head()
df_ins.tail()

In [ ]:
# sample( )의 활용
df_ins.sample(frac=0.005)
df_ins.sample(n=10)

In [ ]:
# nlargest( ), nsmallest( )로 상위/하위 관측치 선택
df_ins.nlargest(10, 'charges')


In [ ]:
df_ins.nsmallest(10, 'charges')

<br>

#### [실습]

1. df_sp에서 math score 상위 20 명 선택
2. df_sp에서 writing score 하위 10명 선택


In [ ]:
df_sp.sample(n=10)

<br>

### 3.7. 중복값 제거

`drop_duplicates()`를 활용해서 중복값을 제거한 목록 생성 가능

In [ ]:
df_ins[['sex','region']].drop_duplicates()

### 3.8. 관측치 정렬

`sort_values()`를 활용해서 관측치를 정렬

In [ ]:
# age 순 데이터 정렬
df_ins.sort_values('age')

In [ ]:
# 원본 데이터는 영향 없음
df_ins.head()

In [ ]:
# 원본 데이터의 정렬
df_ins = df_ins.sort_values('age')
df_ins.head()

In [ ]:
# 내림차순 지정
df_ins = df_ins.sort_values('age', ascending=False)
df_ins.head()

In [ ]:
# 복수 기준의 설정 
df_ins.sort_values(['age', 'charges'], ascending=[True, False])

In [ ]:
# index를 활용한 정렬
df_ins = df_ins.sort_index()
df_ins.head()

<br>


#### [실습] 데이터 df_sp 활용

1. 전체 관측치를 'math score', 'reading score'의 내림차순으로 정렬해서 출력


#### End of script